In [ ]:
import ndjson
import os
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter


# Read NDJSON file with keypoints recorded by Labelbox
ndjson_file = "../../ground_truth/Indoor/court_keypoints/Indoor.ndjson"  # NDJSON file path
with open(ndjson_file, "r") as f:
    data = ndjson.load(f)

# Key point names and corresponding court coordinate system coordinates
keypoint_mapping = {
    "key_1_tokoha": [0, 0],         
    "key_6_tokoha": [0, 1505],      
    "key_9_tokoha": [950, 0],       
    "key_12_tokoha": [950, 1505]    
}

# Extract key point coordinates on the image from NDJSON
points_image = []
points_court = []

annotations = data[0]["projects"]["clqqo8sg92kjv07yx56yad247"]["labels"][0]["annotations"]["objects"]
for annotation in annotations:
    key_name = annotation["value"]
    if key_name in keypoint_mapping:
        x, y = annotation["point"]["x"], annotation["point"]["y"]
        points_image.append([x, y])
        points_court.append(keypoint_mapping[key_name]) 

# Convert to numpy array
points_image = np.array(points_image, dtype=np.float32)
points_court = np.array(points_court, dtype=np.float32)

# Calculate homography matrix
H, status = cv2.findHomography(points_image, points_court)

print("Homography Matrix:\n", H)

Homography Matrix:
 [[ 1.00270933e+00 -8.80391018e+00  2.53903005e+03]
 [-1.93944985e+00 -8.91569008e+00  4.07722989e+03]
 [-1.58145426e-04 -5.04286435e-03  1.00000000e+00]]


In [5]:
def homographic_transformation(x, y, H):
    """Transform the coordinates on the image into the Court coordinate system by homographic transformation"""
    point = np.array([x, y, 1.0]).reshape(3, 1)  # Extended for homographic conversion
    transformed_point = np.dot(H, point)
    transformed_point /= transformed_point[2]  # Normalization
    return transformed_point[0][0], transformed_point[1][0]  # (x', y')

# Court range (using keypoint_mapping)
min_x, min_y = keypoint_mapping["key_1_tokoha"]
max_x, max_y = keypoint_mapping["key_12_tokoha"]
max_x += 100  # give someone ample space

In [ ]:
# Input folder (BoT-SORT output)
input_dir = "../../BoT-SORT_outputs/Indoor"

# Output folder (court coordinate data)
output_dir_court = "../../BoT-SORT_outputs/Indoor/transformed/"
os.makedirs(output_dir_court, exist_ok=True)

# Output folder (filtered bbox data)
output_dir_bbox = "../../BoT-SORT_outputs/Indoor/filtered/"
os.makedirs(output_dir_bbox, exist_ok=True)

# Get MOT files
mot_files = glob.glob(os.path.join(input_dir, "*.txt"))


for mot_file in mot_files:
    
    # Read MOT files
    df = pd.read_csv(
        mot_file,
        header=None,
        names=["frame_id", "id", "x", "y", "width", "height", "conf", "class", "visibility", "empty"],
        sep=",",
    )

    # Calculate the midpoint of the bottom edge of bbox
    df["bottom_center_x"] = df["x"] + df["width"] / 2
    df["bottom_center_y"] = df["y"] + df["height"]

    # Homography transformation of the midpoint of the lower edge of each bbox
    transformed_coords = df.apply(
        lambda row: homographic_transformation(row["bottom_center_x"], row["bottom_center_y"], H),
        axis=1
    )

    # Save the transformed coordinates in a new column
    df["court_x"] = transformed_coords.map(lambda coord: coord[0])
    df["court_y"] = transformed_coords.map(lambda coord: coord[1])

    # Delete bbox outside of court range
    df_court = df[(df["court_x"] >= min_x) & (df["court_x"] <= max_x) & 
                  (df["court_y"] >= min_y) & (df["court_y"] <= max_y)].copy()
    
    # Filter rows for each frame with 7 or more bounding boxes
    df_filtered = df_court.groupby("frame_id").filter(lambda group: len(group) >= 7)

    # Process each frame with 7 or more bounding boxes
    for frame_id, group in df_filtered.groupby("frame_id"):
        # Sort by the x coordinate (court_x) and select the top 6
        sorted_group = group.sort_values(by="court_x", ascending=False).iloc[:6]

        # Replace the original group with the sorted and limited group
        df_court.loc[df_court["frame_id"] == frame_id, :] = sorted_group
    
    df_court = df_court.dropna() 
    
    df_court["frame_id"] = df_court["frame_id"].astype(int) # Convert to integer
    df_court["id"] = df_court["id"].astype(int) # Convert to integer
    
    # bbox data in original MOT format
    df_bbox = df_court[["frame_id", "id", "x", "y", "width", "height", "conf", "class", "visibility", "empty"]]

    # Data of court coordinates (bbox coordinates are deleted and court coordinates are retained)
    df_court = df_court[["frame_id", "id", "court_x", "court_y"]]

    # keep relative paths of files
    relative_path = os.path.relpath(mot_file, input_dir)
    
    # Save converted MOT data (court coordinates)
    output_file_court = os.path.join(output_dir_court, relative_path)
    os.makedirs(os.path.dirname(output_file_court), exist_ok=True)
    df_court.to_csv(output_file_court, index=False, header=None, sep=",")  # Save in MOT format

    # Save converted MOT data (original bbox format)
    output_file_bbox = os.path.join(output_dir_bbox, relative_path)
    os.makedirs(os.path.dirname(output_file_bbox), exist_ok=True)
    df_bbox.to_csv(output_file_bbox, index=False, header=None, sep=",")  # Save in MOT format

    print(f"Processed: {mot_file} -> {output_file_court}, {output_file_bbox}")

print(f"Convert all MOT files, saved in {output_dir_court} (court coordinates) and {output_dir_bbox} (bboxes data).")


Processed: ../BoT-SORT_outputs/Indoor/basket_S6T2_post.txt -> ../BoT-SORT_outputs/Indoor/transformed/basket_S6T2_post.txt, ../BoT-SORT_outputs/Indoor/filtered/basket_S6T2_post.txt
Processed: ../BoT-SORT_outputs/Indoor/basket_S6T3_post.txt -> ../BoT-SORT_outputs/Indoor/transformed/basket_S6T3_post.txt, ../BoT-SORT_outputs/Indoor/filtered/basket_S6T3_post.txt
Processed: ../BoT-SORT_outputs/Indoor/basket_S6T4_post.txt -> ../BoT-SORT_outputs/Indoor/transformed/basket_S6T4_post.txt, ../BoT-SORT_outputs/Indoor/filtered/basket_S6T4_post.txt
Processed: ../BoT-SORT_outputs/Indoor/basket_S6T5_post.txt -> ../BoT-SORT_outputs/Indoor/transformed/basket_S6T5_post.txt, ../BoT-SORT_outputs/Indoor/filtered/basket_S6T5_post.txt
Processed: ../BoT-SORT_outputs/Indoor/basket_S6T6_post.txt -> ../BoT-SORT_outputs/Indoor/transformed/basket_S6T6_post.txt, ../BoT-SORT_outputs/Indoor/filtered/basket_S6T6_post.txt
Processed: ../BoT-SORT_outputs/Indoor/basket_S6T7_post.txt -> ../BoT-SORT_outputs/Indoor/transformed